In [1]:
import pandas as pd
import gzip as gz
import matplotlib as plt
import sqlite3
import os
import zipfile

In [2]:
# Opened the relevant datasets to be explored in jupyter notebook
#if not os.path.exists('zippedData/im.db'):
with zipfile.ZipFile('zippedData/im.db.zip') as my_zip:
    zipfile.ZipFile.extractall(my_zip,path='zippedData/')
with gz.open('zippedData/rt.reviews.tsv.gz') as f:
    rt_reviews = pd.read_csv(f,delimiter='\t',encoding='latin1')
with gz.open('zippedData/rt.movie_info.tsv.gz') as f:
    rt_movie_info = pd.read_csv(f,delimiter='\t',encoding='latin1')
with gz.open('zippedData/tmdb.movies.csv.gz') as f:
    tmdb_movies = pd.read_csv(f)
with gz.open('zippedData/tn.movie_budgets.csv.gz') as f:
    tn_movie_budgets = pd.read_csv(f)
with gz.open('zippedData/bom.movie_gross.csv.gz') as f:
    bom_movie_gross = pd.read_csv(f)

In [3]:
# Connected to and opened the imdb database to utilize the data
im_db = sqlite3.connect('zippedData/im.db')
imdb_df = pd.read_sql("""SELECT * FROM movie_basics""",im_db)

In [4]:
imdb_df.value_counts(['primary_title'])
imdb_df[imdb_df['primary_title'] == 'Home']
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [5]:
tn_movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [6]:
bom_movie_gross.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [7]:
imdb_df.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [8]:
tn_movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [9]:
bom_movie_gross.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [10]:
# Merged the imdb + tn_movie_budgets and imdb + bom_movie_gross datasets as two seperate merged datasets, then merged
# The two merged datasets into a final dataset called all_movie_data
imdb_and_bom_data = imdb_df.merge(bom_movie_gross, left_on = 'primary_title', right_on = 'title')
imdb_and_tn_data = imdb_df.merge(tn_movie_budgets, left_on = 'primary_title', right_on = 'movie')
all_movie_data = imdb_and_bom_data.merge(imdb_and_tn_data, left_on = 'movie_id', right_on = 'movie_id')
all_movie_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1602 entries, 0 to 1601
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   movie_id           1602 non-null   object 
 1   primary_title_x    1602 non-null   object 
 2   original_title_x   1602 non-null   object 
 3   start_year_x       1602 non-null   int64  
 4   runtime_minutes_x  1502 non-null   float64
 5   genres_x           1576 non-null   object 
 6   title              1602 non-null   object 
 7   studio             1602 non-null   object 
 8   domestic_gross_x   1601 non-null   float64
 9   foreign_gross      1372 non-null   object 
 10  year               1602 non-null   int64  
 11  primary_title_y    1602 non-null   object 
 12  original_title_y   1602 non-null   object 
 13  start_year_y       1602 non-null   int64  
 14  runtime_minutes_y  1502 non-null   float64
 15  genres_y           1576 non-null   object 
 16  id                 1602 

In [11]:
# dropped the repeated column names as well as the columns that were not relevant to the business question.
all_movie_data.drop(['original_title_x', 'start_year_x', 'runtime_minutes_x', 'domestic_gross_x', 'foreign_gross', 'movie',\
                    'original_title_y', 'runtime_minutes_y', 'id', 'primary_title_x','start_year_y','primary_title_y',\
                    'genres_y'], axis = 1, inplace = True)

In [12]:
# Using this to check how many titles have more than one value - meaning it is likely a repeated entry
all_movie_data['title'].value_counts().reset_index(name="count").query("count > 1")


,index,count
0,The Journey,11
1,The Wall,10
2,The Gambler,10
3,Robin Hood,10
4,Eden,9
...,...,...
198,Project X,2
199,Wolves,2
200,The Three Stooges,2
201,Tomorrowland,2


In [13]:
all_movie_data[all_movie_data['title'] == 'The Losers']

,movie_id,genres_x,title,studio,year,release_date,production_budget,domestic_gross_y,worldwide_gross
50,tt0480255,"Action,Adventure,Crime",The Losers,WB,2010,"Apr 23, 2010","$25,000,000","$23,591,432","$29,863,840"
51,tt3746918,Drama,The Losers,WB,2010,"Apr 23, 2010","$25,000,000","$23,591,432","$29,863,840"
